<a href="https://colab.research.google.com/github/minamh9/Prompt-Engineering-with-GPT/blob/main/Fewshot_GPT_Amazon_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.0 MB/s eta 0:00:00


In [ ]:
import os
from pathlib import Path
import openai
import requests
import uuid
import pandas as pd
import tiktoken
import sys
import gzip
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error


In [ ]:
from config import OPENAI_KEY

In [ ]:
if OPENAI_KEY:
  print("OpenAI Key loaded successfully!")
else:
  print("Failed to load OpenAI Key!")


OpenAI Key loaded successfully!


In [ ]:
openai.api_key = OPENAI_KEY

In [ ]:
# Load dataset from JSON gzip file
file_path = '/content/drive/MyDrive/AMAZON_FASHION.json.gz'

with gzip.open(file_path) as f:
  df = pd.read_json(f, lines = True, nrows= 100000)

In [ ]:
#keep only few colmns that we need
df = df[['overall','verified','reviewTime','reviewText']]

#drop the non-verified reviews, and reset the DataFrame's index after filtering rows
df = df.drop(df[df['verified']==False].index).reset_index(drop = True)

#Remove rows with missing values
df = df.dropna(subset=['reviewText', 'overall'])

# Adjust the 'overall' column values to convert a 5-star rating system to a 0-4 scale
df['overall'] = df['overall'] - 1

# Extract the first 1,000 rows for testing and reset their index
df_test = df.iloc[:1000, :].reset_index(drop=True)

<ipython-input-47-602c57a1dd21>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overall'] = df['overall'] - 1


In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0 # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
prompt = f"""
Your task is to evaluate the following review, providing a score from 0 to 4. \
Note that 0 denotes a poor review while 4 signifies an excellent one.\
Your response should simply be a single integer between 0 and 4. \
If the review references any photographs or similar elements, please disregard them. \

<review>: Velcro backs are no good. They don't stay on. Look old after one use.
 0

<review>: Really cheap looking for the price and the thing in the middle fell off after one day.
1

<review>: Super cute but way too big for infants!
2

<review>: Great costume to wear once or twice, but the stitching doesn't hold up to extended use. Very pleased with it.
3

<review>: Bought this for my sons blessing outfit. He looked super handsome in it. I pinned a little bow tie to add some color and it was perfect. The color is actually super white. Material is soft and comfy. I ordered a 3 month size and he wore it at 6 weeks. Was worried it would be too big but it fit perfectly.
4

<review>: \"\"\"{df_test['reviewText'][25]}\"\"\"
"""
response = get_completion(prompt)
print(response)

1


In [ ]:
fewshot_responses = []
for text in df_test['reviewText'].iloc[:100]:
    prompt = f"""
    Your task is to evaluate the following review, providing a score from 0 to 4. \
    Note that 0 denotes a poor review while 4 signifies an excellent one.\
    Your response should simply be a single integer between 0 and 4. \
    If the review references any photographs or similar elements, please disregard them. \

    <review>: I bought 2 pairs of these socks in brown and 3 pairs in charcoal. I had to send back 1 pair of the charcoal because holes had formed in the heel even though I only use them for an 8 hour period and then hand wash same.  Now, I will need to send back 1 pair of the brown socks because a hole formed in one of the heels.  I only wear these socks at home with slippers and can't for the life of me understand why they are wearing out so quickly, but it seems to be a problem with durability/quality.
    0

    <review>:These tights were super cute and perfect for my witch costume, sadly the crotch of the tights only hit right above my knee. It was so close to Halloween that I cut them off and wore them as knee highs. This only worked for about 20 minutes. Then they started to roll down.
    1

    <review>: Super cute but way too big for infants!
    2

    <review>: Great costume to wear once or twice, but the stitching doesn't hold up to extended use. Very pleased with it.
    3

    <review>: Bought this for my sons blessing outfit. He looked super handsome in it. I pinned a little bow tie to add some color and it was perfect. The color is actually super white. Material is soft and comfy. I ordered a 3 month size and he wore it at 6 weeks. Was worried it would be too big but it fit perfectly.
    4
    <review>: \"\"\"{text}\"\"\"
    """

    response = get_completion(prompt)
    fewshot_responses.append(response)

    time.sleep(10)
len(fewshot_responses)

100

In [ ]:
fewshot_responses = []

In [ ]:
for text in df_test['reviewText'].iloc[900:1000]:
    prompt = f"""
    Your task is to evaluate the following review, providing a score from 0 to 4. \
    Note that 0 denotes a poor review while 4 signifies an excellent one.\
    Your response should simply be a single integer between 0 and 4. \
    If the review references any photographs or similar elements, please disregard them. \

    <review>: I bought 2 pairs of these socks in brown and 3 pairs in charcoal. I had to send back 1 pair of the charcoal because holes had formed in the heel even though I only use them for an 8 hour period and then hand wash same.  Now, I will need to send back 1 pair of the brown socks because a hole formed in one of the heels.  I only wear these socks at home with slippers and can't for the life of me understand why they are wearing out so quickly, but it seems to be a problem with durability/quality.
    0

    <review>:These tights were super cute and perfect for my witch costume, sadly the crotch of the tights only hit right above my knee. It was so close to Halloween that I cut them off and wore them as knee highs. This only worked for about 20 minutes. Then they started to roll down.
    1

    <review>: Super cute but way too big for infants!
    2

    <review>: Great costume to wear once or twice, but the stitching doesn't hold up to extended use. Very pleased with it.
    3

    <review>: Bought this for my sons blessing outfit. He looked super handsome in it. I pinned a little bow tie to add some color and it was perfect. The color is actually super white. Material is soft and comfy. I ordered a 3 month size and he wore it at 6 weeks. Was worried it would be too big but it fit perfectly.
    4
    <review>: \"\"\"{text}\"\"\"
    """

    response = get_completion(prompt)
    fewshot_responses.append(response)

    time.sleep(10)
len(fewshot_responses)

400

In [ ]:
df_test.loc[600:999,'fewShot_GPT_pred'] = fewshot_responses
df_test.to_csv('./fewshot_gpt_results_part2.csv', index=False)

In [ ]:
df_test = pd.read_csv('fewshot_gpt_result.csv')

In [ ]:
df_test

,overall,verified,reviewTime,reviewText,fewShot_GPT_pred
0,4,True,"10 20, 2014",Exactly what I needed.,3
1,1,True,"09 28, 2014","I agree with the other review, the opening is ...",1
2,1,True,"08 24, 2014",too tiny an opening,1
3,4,True,"07 19, 2014",Exactly what I wanted.,3
4,3,True,"05 31, 2014",These little plastic backs work great. No mor...,3
...,...,...,...,...,...
995,0,True,"09 14, 2014","Didn't order, must have hit by mistake, had to...",2
996,2,True,"08 9, 2014",I bought these for a Halloween costume and for...,2
997,2,True,"05 14, 2014","The product is just okay for me, it would have...",2
998,4,True,"04 21, 2014",I already had a pair of these that I have owne...,4


In [ ]:
# Create a dataframe to compare true vs predicted values
result_df = pd.DataFrame({'True': df_test['overall'], 'Predicted': df_test['fewShot_GPT_pred'],'Review': df_test['reviewText']})
result_df

,True,Predicted,Review
0,4,3,Exactly what I needed.
1,1,1,"I agree with the other review, the opening is ..."
2,1,1,too tiny an opening
3,4,3,Exactly what I wanted.
4,3,3,These little plastic backs work great. No mor...
...,...,...,...
995,0,2,"Didn't order, must have hit by mistake, had to..."
996,2,2,I bought these for a Halloween costume and for...
997,2,2,"The product is just okay for me, it would have..."
998,4,4,I already had a pair of these that I have owne...


In [ ]:
# Compute evaluation metrics to assess model performance
accuracy = accuracy_score(result_df['True'], result_df['Predicted'])
print("Accuracy:", accuracy)

precision = precision_score(result_df['True'], result_df['Predicted'], average='weighted')
print("Precision:", precision)

recall = recall_score(result_df['True'], result_df['Predicted'], average='weighted')
print("Recall: ", recall)

f1 = f1_score(result_df['True'], result_df['Predicted'], average='weighted')
print("F1_score:" , f1)

mae = mean_absolute_error(result_df['True'], result_df['Predicted'])
print("mae" , mae)

Accuracy: 0.35
Precision: 0.6333306906782257
Recall:  0.344
F1_score: 0.3079192146343119
mae 0.804


In [ ]:
df_train = df[10001:]

In [ ]:
print(df_train[df_train['overall']==1].sample()['reviewText'].values)

['The lens quality is ok for short periods of use.  For any long periods of reading the quality feels slightly distorted and puts a strain on your eyes.']
